In [2]:
!pip install hdfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 262.4 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hdfs: filename=hdfs-2.7.3-py3-none-any.whl size=34325 sha256=932b726aab982d5e9a24f1e79e5dde37f306f7020babd6665324ba7e432674b8
  Stored in directory: /home/jovyan/.cache/pip/wheels/b9/1d/dc/eb0833be25464c359903d356c4204721c6a672c26ff164cdc3
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=3147df1b7b3788b977130dee8db66bb563ba66ea14d6978124223f0fa7c8a77c
  Stored in directory: /home/jovyan/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built hdfs docopt


In [ ]:
!pip install tensorflow

   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/620.6 MB 8.3 MB/s eta 0:00:59m

In [ ]:
from hdfs import InsecureClient
from tensorflow import keras
from tensorflow.keras import losses, metrics
import pandas as pd


In [25]:
# Establish connection to HDFS
connection_url = "http://localhost:9870"
hdfs_client = InsecureClient(connection_url, user='root')
hdfs_client.status('/')

{'accessTime': 0,
 'blockSize': 0,
 'childrenNum': 1,
 'fileId': 16385,
 'group': 'supergroup',
 'length': 0,
 'modificationTime': 1770282148988,
 'owner': 'root',
 'pathSuffix': '',
 'permission': '755',
 'replication': 0,
 'snapshotEnabled': True,
 'storagePolicy': 0,
 'type': 'DIRECTORY'}

In [26]:
# Load Model
model_path = "/home/pooya/Desktop/DSFP/models/weather_lstm_model.h5"

custom_objects = {
    'mse': losses.MeanSquaredError(),
    'mae': metrics.MeanAbsoluteError()
}

model = keras.models.load_model(
    model_path,
    custom_objects=custom_objects,
    compile=False  # Don't compile during load
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=losses.MeanSquaredError(),
    metrics=[metrics.MeanAbsoluteError()]
)

In [27]:
raw_path = '/weather/raw'

locations = []
for item in hdfs_client.list(raw_path):
    if item.startswith('location='):
        location = item.replace('location=', '')
        locations.append(location)

locations

['Albuquerque',
 'Atlanta',
 'Beersheba',
 'Boston',
 'Charlotte',
 'Chicago',
 'Dallas',
 'Denver',
 'Detroit',
 'Eilat',
 'Haifa',
 'Houston',
 'Indianapolis',
 'Jacksonville',
 'Jerusalem',
 'Kansas City',
 'Las Vegas',
 'Los Angeles',
 'Miami',
 'Minneapolis',
 'Montreal',
 'Nahariyya',
 'Nashville',
 'New York',
 'Philadelphia',
 'Phoenix',
 'Pittsburgh',
 'Portland',
 'Saint Louis',
 'San Antonio',
 'San Diego',
 'San Francisco',
 'Seattle',
 'Tel Aviv District',
 'Toronto',
 'Vancouver']

In [ ]:
def fetch_recent_data(location, hours=48):
    # Path to location partition
    partition_path = f'/weather/raw/location={location}'
    
    # Read parquet files
    parquet_files = []
    for file in hdfs_client.list(partition_path):
        if file.endswith('.parquet'):
            file_path = f'{partition_path}/{file}'
            parquet_files.append(file_path)


    import pyarrow as pa
    import pyarrow.parquet as pq

    fs = pa.connect(host='localhost', port=9005, user='root')

    dfs = []
    for file_path in parquet_files:
        table = pq.read_table(file_path, filesystem=fs)
        df = table.to_pandas()
        dfs.append(df)

    # Problem:
    # with hdfs_client.read(parquet_files[0]) as f:
    #     print(f)

    return parquet_files    # For debugging purposes -> works fine
    
    # # Read all parquet files
    dfs = []
    for file_path in parquet_files:
        with hdfs_client.read(file_path) as reader:
            df = pd.read_parquet(reader)
            dfs.append(df)

    # return dfs    # For debugging purposes -> problematic
    
    # # Combine all dataframes
    # combined_df = pd.concat(dfs, ignore_index=True)
    
    # # Convert timestamp to datetime
    # combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'])
    
    # # Sort by timestamp
    # combined_df = combined_df.sort_values('timestamp')
    
    # # Filter to recent hours
    # cutoff_time = datetime.now() - timedelta(hours=hours)
    # recent_df = combined_df[combined_df['timestamp'] >= cutoff_time]
    
    # logger.info(f"Fetched {len(recent_df)} records for {location}")
    # return recent_df

In [33]:
fetch_recent_data('Albuquerque')

AttributeError: module 'pyarrow' has no attribute 'hdfs'